In [49]:
import pandas as pd
from openpyxl import load_workbook

In [50]:
excel_path = "scheme.xlsx"
sheet_name = "Sheet1"
latex_path = "scheme.tex"

In [51]:
wb = load_workbook(excel_path)
sheet = wb[sheet_name]

In [52]:
sheet.merged_cells.ranges

{<MergedCellRange B5:B8>,
 <MergedCellRange B17:B19>,
 <MergedCellRange A17:A29>,
 <MergedCellRange B12:B14>,
 <MergedCellRange B20:B23>,
 <MergedCellRange B9:B11>,
 <MergedCellRange A2:A14>,
 <MergedCellRange B24:B26>,
 <MergedCellRange B2:B4>,
 <MergedCellRange B27:B29>}

In [53]:
def escape(s: str):
    return s.replace("_", "\\_")

In [54]:
# latex = "\\begin{tabular}{" + "l" * sheet.max_column + "}\n\\hline\n"
latex = "\\begin{tabular}{p{1.6cm}||p{1.7cm}|p{2cm}p{10.8cm}}"
latex += "\\hline\n"

for row in sheet.iter_rows():

    cells_latex = []
    cline_ranges = []

    for cell in row:

        # Find merged range
        is_merged, is_merged_master, is_merged_tail = False, False, False
        for merged_range in sheet.merged_cells.ranges:
            if cell.coordinate in merged_range:
                is_merged = True
                is_merged_master = is_merged and cell.value is not None
                is_merged_tail = is_merged and cell.column == merged_range.max_col and cell.row == merged_range.max_row
                break

        cell_value = escape(str(cell.value)) if cell.value else ""
        cell_value = f"\\textbf{{{cell_value}}}" if cell.font.bold else cell_value
        cell_value = f"\\underline{{{cell_value}}}" if cell.font.underline else cell_value
        cell_value = f"\\textit{{{cell_value}}}" if cell.font.italic else cell_value

        if is_merged and is_merged_master:
            merged_size_columns = merged_range.max_col - merged_range.min_col + 1
            merged_size_rows = merged_range.max_row - merged_range.min_row + 1
            cells_latex.append(
                f"\\multirow{{{merged_size_rows}}}{{*}}{{{cell_value}}}"
            )

        elif is_merged:
            cells_latex.append("")
        else:
            cells_latex.append(f"{cell_value}")

        if not is_merged:
            cline_ranges.append(cell.column)
        elif is_merged_tail:
            cline_ranges.append(cell.column)

    latex += " & ".join(cells_latex) + " \\\\\n"
    latex += "".join(
        map(lambda x: f"\\cline{{{str(x)}-{str(x)}}}", set(cline_ranges))
    )
    latex += "\n"

latex += "\\hline\n"
latex += "\\end{tabular}"

In [55]:
cline_ranges

[1, 2, 3, 4, 5]

In [56]:
# Save the LaTeX code to a .tex file
with open(latex_path, "w") as f:
    f.write(latex)